<a href="https://colab.research.google.com/github/LW068/atlas-machine_learning/blob/main/reinforcement_learning/deep_q_learning/deep-q-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations & imports

In [1]:
import os
import tensorflow as tf

In [2]:
!pip install keras-rl2 --quiet
!pip install gym[atari] --quiet
!pip install atari-py --quiet
!pip install tensorflow==2.11.0 --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!python -m atari_py.import_roms /content/drive/MyDrive/dqn/roms/

copying breakout.bin from /content/drive/MyDrive/dqn/roms/Breakout - Breakaway IV.bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/breakout.bin


In [5]:
import gym
from gym.envs.registration import register

/usr/local/lib/python3.10/dist-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()


# Registering

In [6]:
register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment Breakout-v4
  logger.warn(f"Overriding environment {spec.id}")


# All needed imports (might not use all)

In [7]:
import tensorflow as tf
import gym
import numpy as np
import keras.backend as K

from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from keras.callbacks import ModelCheckpoint

In [8]:
!pip install gym[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=1092f9da77ad4b0a6cf29fc3068dfaf6196851759e9adf352a2091cfc87ae995
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


# Environment

In [9]:
environment = gym.make('Breakout-v4')
np.random.seed(123)
environment.seed(123)
action_size = environment.action_space.n

/usr/local/lib/python3.10/dist-packages/gym/envs/atari/environment.py:68: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


# Building Neural Network

In [10]:
neural_model = Sequential()
neural_model.add(Flatten(input_shape=(1,) + environment.observation_space.shape))
neural_model.add(Dense(40, activation='relu'))
neural_model.add(Dense(30, activation='relu'))
neural_model.add(Dense(action_size, activation='linear'))

# Compiling

In [11]:
memory_buffer = SequentialMemory(limit=50000, window_length=1)
policy_choice = EpsGreedyQPolicy()
dqn_agent = DQNAgent(model=neural_model, nb_actions=action_size, memory=memory_buffer, nb_steps_warmup=100,
                     target_model_update=1e-2, policy=policy_choice)
dqn_agent.compile(Adam(learning_rate=1e-3), metrics=['mae'])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


# Print Summary

In [12]:
print(neural_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 100800)            0         
                                                                 
 dense (Dense)               (None, 40)                4032040   
                                                                 
 dense_1 (Dense)             (None, 30)                1230      
                                                                 
 dense_2 (Dense)             (None, 4)                 124       
                                                                 
Total params: 4,033,394
Trainable params: 4,033,394
Non-trainable params: 0
_________________________________________________________________
None


# Training - ~10 mins (2,400 steps)

In [13]:
from keras.callbacks import ModelCheckpoint


# defnining a checkpoint callback
checkpoint = ModelCheckpoint('policy_checkpoint.h5', monitor='episode_reward', save_best_only=True, verbose=2)

# training the agent
dqn_agent.fit(environment, nb_steps=2400, visualize=False, verbose=2, callbacks=[checkpoint])

# saving the final policy h5 network
dqn_agent.save_weights('policy.h5', overwrite=True)


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Training for 2400 steps ...


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


AttributeError: 'DQNAgent' object has no attribute 'distribute_strategy'

# Play.py task

In [ ]:
# loading  the trained weights
model.load_weights('policy.h5')

# configuring the agent with "GreedyQPolicy"
memory = SequentialMemory(limit=50000, window_length=1)
policy = GreedyQPolicy()
dqn_agent = DQNAgent(model=model, nb_actions=action_size, memory=memory, policy=policy)

# compiling agent
dqn_agent.compile()

# testing agent
dqn_agent.test(environment, nb_episodes=5, visualize=True)